# PRICING DINÁMICO DE HABITACIONES DE HOTEL

*Cristian Rubio Barato*

*Francisco Martínez Esteso*

*José Vicente García López*

*Víctor Ortega Gómez*


## ÍNDICE

1. [ANÁLISIS EXPLORATORIO DE DATOS](#1-análisis-exploratorio-de-datos)  
   1.1 [Tipos de Variables y Distribución](#11-tipos-de-variables-y-distribución)  
   1.2 [Análisis Univariado](#12-análisis-univariado)  
   1.3 [Análisis Multivariado](#13-análisis-multivariado)  
   1.4 [Correlación de Variables Continuas](#14-correlación-de-variables-continuas)  
   1.5 [Importancia de Variables Continuas](#15-importancia-de-variables-continuas)  

2. [PREPROCESAMIENTO](#2-preprocesamiento)  
   2.1 [Limpieza de los datos](#21-limpieza-de-los-datos)  
   2.2 [Pipeline de Datos](#22-pipeline-de-datos)

## 1. ANÁLISIS EXPLORATORIO DE DATOS

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('hotel_booking.csv')

categorical_vars = df.select_dtypes(include=['object', 'category']).columns.tolist()
continuous_vars = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

print("Variables Categóricas:", categorical_vars)
print("Variables Continuas:", continuous_vars)

Variables Categóricas: ['hotel', 'arrival_date_month', 'meal', 'country', 'market_segment', 'distribution_channel', 'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type', 'reservation_status', 'reservation_status_date', 'name', 'email', 'phone-number', 'credit_card']
Variables Continuas: ['is_canceled', 'lead_time', 'arrival_date_year', 'arrival_date_week_number', 'arrival_date_day_of_month', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', 'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled', 'booking_changes', 'agent', 'company', 'days_in_waiting_list', 'adr', 'required_car_parking_spaces', 'total_of_special_requests']


### 1.1 Tipos de Variables y Distribución

### 1.2 Análisis Univariado

### 1.3 Análisis Multivariado

### 1.4 Correlación de Variables Continuas

### 1.5 Importancia de Vriables Continuas

## 2. PREPROCESAMIENTO

### 2.1 Limpieza de los datos

### 2.2 Pipeline de Datos